## Relevant Changes
Run this script on all 3 data sets  
Sample 25 seeds and calculate a rank for every variable  
store these ranks and compute the average score at the end of 25 trials  
plot these ranks and their standard errors  

# Adamy Valuation - Michigan Data Science Team
### Michael Xinyu Tim Sid Derek Manny
                                                                                                        5/06/2017

**Purpose:** This notebook describes the data processing pipeline we have built and instructions on how to interpret the results.  
  
**Conclusion:** Ebitda/interest_exp is the best predictor of ev/ebidta. It seems that the higher the ebitda/interest_exp the lower our prediction for ev/ebitda. Other important features are ebitda_margin and ebitda.  
  
**Notes:** It was hard to precisely quantify the degree to which these variables actually impacted ev/ebitda because of the limitations of linear regression. We have provided several lists of features that our models found to be important. It would be interesting to compare and contrast these lists with the existing literature. 

## Load Libraries

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from get_data import *

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import scipy.stats as stats
import random
from mpl_toolkits.mplot3d import Axes3D
from get_data import *

# SciKit Learn Modules
from sklearn.preprocessing import scale
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor

## Load Data

In [4]:
np.random.seed(0)
raw_data_frame = load_data_frames()
sectors = ['consumerDiscrete', 'consumerStaples', 'industrials']

Our exploratory data anlaysis led us to discover that our models would be more effective after transforming some of the columns. Specifically those columns that represent monetary values as well as a few others.

In [5]:
for i in range(len(raw_data_frame[0].columns)):
    print(i, raw_data_frame[0].columns[i])

(0, 't_ev')
(1, 't_rev')
(2, 'ebitda')
(3, 'ebitda_margin')
(4, 't_ev/t_rev')
(5, 't_ev/ebitda')
(6, 'est_ann_rev_gr_1yr')
(7, 'est_ann_ebitda_gr_1yr')
(8, 't_rev_1yr_growth')
(9, 'ebitda_1yr_growth')
(10, 't_rev_3_yr_cagr')
(11, 'ebitda_3yr_cagr')
(12, 't_rev_5yr_cagr')
(13, 'ebitda_5yr_cagr')
(14, 'return_on_assets')
(15, 'return_on_equity')
(16, 'capex_as_percent_rev')
(17, 'ebitda/interest_exp')
(18, 't_debt/cap_percent')
(19, 't_debt/equity_percent')


In [6]:
log_transform_columns =  [0, 1, 2, 5, 16, 17, 18, 19]
log_transform_col_names = []
for col in log_transform_columns:
    log_transform_col_names.append(raw_data_frame[0].columns[col])
print log_transform_col_names

['t_ev', 't_rev', 'ebitda', 't_ev/ebitda', 'capex_as_percent_rev', 'ebitda/interest_exp', 't_debt/cap_percent', 't_debt/equity_percent']


We need to actually modify the data for the columns we discovered.  
**Note:** Because we do the following process in place it is important that the line is not run twice.

In [7]:
# log-transform inplace
for i in range(len(sectors)):
    raw_data_frame[i][log_transform_col_names] = raw_data_frame[i][log_transform_col_names].apply(np.log10)
    raw_data_frame[i].rename(columns = {
        't_ev':'log_t_ev',
        't_rev':'log_t_rev',
        'ebitda':'log_ebitda',
        't_ev/ebitda':'log_t_ev/ebitda',
        'capex_as_percent_rev':'log_capex_as_percent_rev',
        'ebitda/interest_exp':'log_ebitda/interest_exp',
        't_debt/cap_percent':'log_t_debt/cap_percent',
        't_debt/equity_percent':'log_t_debt/equity_percent'
                                },inplace = True)    

In order to handle missing data we fill in cells the with mean for the column

In [8]:
for i in range(len(sectors)):
    raw_data_frame[i].fillna(raw_data_frame[i].mean(), inplace=True)

## Prepare the data for ML models

- Target: 't_ev/ebitda'
- Discard 't_ev'

From pearson correlation heatmap, we see that target variable is not strongly linearly associated with other features. Therefore, the nonlinear methods, like tree-based methods, or SVM with nonlinear kernels will outperform. However, one thing to notice is that our final goal is not to predict the target value but rather to understand the complex relationship between ev/ebitda and other features. We shall try various methods  
  
We chose to discared the feature t_ev and t_ev/t_rev because we believe it will not be known for a private company

In [9]:
# drop 't_ev' columns
for i in range(len(sectors)):
    raw_data_frame[i].drop('log_t_ev', axis=1, inplace=True)
    raw_data_frame[i].drop('t_ev/t_rev', axis=1, inplace=True)

In [10]:
print("Let's look at " + sectors[0])
cur_df = raw_data_frame[0]

Let's look at consumerDiscrete


In [11]:
y = cur_df['log_t_ev/ebitda']
X = cur_df.drop('log_t_ev/ebitda', axis=1)
X.head()

,log_t_rev,log_ebitda,ebitda_margin,est_ann_rev_gr_1yr,est_ann_ebitda_gr_1yr,t_rev_1yr_growth,ebitda_1yr_growth,t_rev_3_yr_cagr,ebitda_3yr_cagr,t_rev_5yr_cagr,ebitda_5yr_cagr,return_on_assets,return_on_equity,log_capex_as_percent_rev,log_ebitda/interest_exp,log_t_debt/cap_percent,log_t_debt/equity_percent
0,3.075182,1.897077,6.64,4.540000,16.700000,2.06000,-0.852000,16.400000,19.400000,11.200000,18.700000,4.600000,9.900000,0.460898,1.103804,1.513218,1.685742
1,2.426511,0.772322,2.22,7.132813,17.742803,-1.93000,13.234825,-4.040000,-34.500000,-10.600000,-28.900000,-1.570000,-12.200000,0.444045,1.004171,1.561461,1.781483
2,3.509740,2.665393,14.30,1.480000,4.270000,4.08000,4.980000,13.000000,9.600000,10.300000,11.200000,9.930000,9.950000,0.278754,1.281033,1.489958,1.650308
3,3.531900,2.415808,7.65,-5.230000,-36.900000,-3.46000,-19.200000,-7.400000,-21.700000,-3.080000,-15.300000,1.560000,1.330000,0.609594,1.060698,1.311754,1.411620
4,3.191898,2.244277,11.30,7.132813,17.742803,331.70347,13.234825,12.670764,8.528391,8.063843,7.665472,6.344578,14.769923,0.187521,0.481443,1.857332,2.411114


It is common practice in Machine Learning to use inputs of mean = 0 and standard deviation = 1. This way the impact of the features can be compared head to head. The scale() function below does this transformation

In [12]:
from sklearn.preprocessing import scale
X_scale = scale(X)

Before we start doing any model selection it is important that we break our data up into a training and a test set. The training set will be used to select and train the models. The test set will come in at the end of the program and will be our way of testing a models accuracy on data that is has never seen.  
The training set will consist of 75% of our data and the test set will have the remaining 25%. Each set has an "X" and "y" component. The y component is what we are trying to predict, t_ev/ebitda. The X component consists of all other variables in the data set (ebitda, ebitda_margin, est_ann_rev_gr_1yr, etc.) which we will be using to predict t_ev/ebitda.

In [13]:
x_train, x_test, y_train, y_test = train_test_split(X_scale,y,train_size = 0.75)

## Random Forest Classifier

In [14]:
# basic model 
model = RandomForestRegressor()
scores = cross_val_score(model, x_train, y_train, scoring='neg_mean_squared_error', cv=5)
# print('MSEs:', scores*-1)
print("Average MSE: %0.4f (+/- %0.4f)" % (-scores.mean(), scores.std()))

Average MSE: 0.0549 (+/- 0.0096)


There are a bunch of 'tuning parameters' for random forest. One commonly used method is to just try a bunch of combinations and take the the one that works best.

In [15]:
from sklearn.ensemble import RandomForestRegressor
from itertools import product
n_estimators_params = [10, 20]
max_features_params = ['auto', 'sqrt', "log2"]
min_samples_split_params = [2]
min_samples_leaf_params = [  2]

result_dict = {}
for param in product(n_estimators_params, max_features_params, min_samples_split_params, min_samples_leaf_params):
    rf_model = RandomForestRegressor(n_estimators=param[0], max_features=param[1],
                                min_samples_split=param[2], min_samples_leaf=param[3], 
                                 random_state=1711, n_jobs=-1)
    scores = cross_val_score(rf_model, x_train, y_train, scoring='neg_mean_squared_error', cv=5)
    result_dict[param] = (-scores.mean(), scores.std())
#     print 'current param: n_estimators_params, max_features_params, min_samples_split_params, min_samples_leaf_params'
#     print param
#     print 'mse mean and std: ',  -scores.mean(), scores.std()

In [ ]:
min_mse = 100
min_param = ()
for param in result_dict:
    if result_dict[param][0] < min_mse:
        min_mse=result_dict[param][0]
        min_param = param
print result_dict[min_param], min_param

(0.052131293839246737, 0.0087931074800553753) (20, 'sqrt', 2, 2)


We found that the optimal model uses 20 estimators, 'auto' feature parameters, and will split a minimum of 2 times and leave a minimum of 2 leafs. It is not useful to go into precise defintions of what these terms means. The implications are not clear. We want to focus on the feature importance that will come out as a result.

In [ ]:
cumulative_winnings = []
np.random.seed(212)
for j in range(100):
    seed = np.random.randint(0,10**4)
    x_train, x_test, y_train, y_test = train_test_split(X_scale,y,train_size = 0.75)
    rf_model = RandomForestRegressor(n_estimators=20, max_features='auto',
                                    min_samples_split=2, min_samples_leaf=2,random_state=seed, n_jobs=-1)

    rf_model.fit(x_train, y_train)
    yhat_rf = rf_model.predict(x_test)
    #print 'Finished!'
    feature_importance = rf_model.feature_importances_
 
    rf_feat_pairs = []

    for i in range(len(feature_importance)):
        rf_feat_pairs.append((X.columns[i],feature_importance[i]))     
        cumulative_winnings.append((X.columns[i],feature_importance[i]))


# Now I need to average over all instances of column to compute avg score    


In [ ]:
df = pd.DataFrame()


for i in range(len(X.columns)):
    rankings = []
    for j in range(len(cumulative_winnings)):
        if X.columns[i] == cumulative_winnings[j][0]:
            rankings.append(cumulative_winnings[j][1])
    df = df.append({'feature': cumulative_winnings[i][0], 'mean_importance': np.mean(rankings), 'se': stats.sem(rankings)}, ignore_index=True)        
#     print(cumulative_winnings[i][0])
#     print(np.mean(rankings))
#     print(np.std(rankings))

df = df.sort("mean_importance",ascending=0)

In [ ]:
print(df)

In [ ]:
plot_df = df[0:17]
se = plot_df["se"] * 2
tuple(se)

In [ ]:
"""
========
Barchart
========

A bar plot with errorbars and height labels on individual bars
"""
import numpy as np
import matplotlib.pyplot as plt

mean_importance = plot_df["mean_importance"]
se = plot_df["se"] * 2

ind = np.arange(len(plot_df))  # the x locations for the groups
width = 0.35       # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(ind, mean_importance, width, color='r', yerr=tuple(se))



# add some text for labels, title and axes ticks
ax.set_ylabel('Average Importance')
ax.set_title('Feature Importance (100 Trials): Consumer Discrete Sector')
plt.xticks(rotation=70)
ax.set_xticks(ind + width/2)
ax.set_xticklabels(plot_df["feature"])


plt.show()